In [1]:
# %pip install astropy\n# %pip install scipy\n# %pip install matplotlib\n# %pip install astroquery\n# %pip install skyfield


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import astropy.io.fits as fits
from astropy.convolution import convolve, Gaussian2DKernel
from astropy.table import Table
from astropy.modeling.models import Moffat2D
import scipy.stats
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta

from synthetic_image import (
    get_star_catalog,
    build_synthetic_wcs,
    world_to_pixel,
    mag_to_flux,
    add_psf_to_image,
    make_gaussian_psf,
    make_moffat_psf,
    sky_brightness_to_electrons,
    add_satellite_substepping,
    add_star_trails_skyfield
)

In [3]:
image_width = 4144
image_height = 2822

ra_center = 88.29533
dec_center = -5.364887
rotation_degrees = -52.0

max_mag = 19.5

seeing_arcsec = 5.5
pixscale_arcsec = .495

aperture_area      = 324.29      # cm^2
quantum_efficiency = 0.800
exposure_time      = 5.0     # seconds
sky_mag_per_arcsec2 = 18.5    # dark-ish sky
mag_zero_point     = 15.5

In [4]:
wcs_obj, wcs_header = build_synthetic_wcs(
    image_width,
    image_height,
    ra_center,
    dec_center,
    pixscale_arcsec=pixscale_arcsec,
    rotation_degrees=rotation_degrees
)

In [5]:
synthetic_image = np.zeros((image_height, image_width), dtype=float)

In [6]:
fwhm_pixels = seeing_arcsec / pixscale_arcsec

In [7]:
psf_kernel = make_moffat_psf(
    size=int(fwhm_pixels * 6),
    fwhm=fwhm_pixels,
    moffat_alpha=4
)
plt.imshow(psf_kernel, cmap='gray')
plt.colorbar()
plt.show()

In [8]:
sky_bg = sky_brightness_to_electrons(
    sky_mag_per_arcsec2,
    plate_scale_arcsec_per_pix=.495,
    exposure_time=exposure_time,
    quantum_efficiency=quantum_efficiency,
    aperture_area=aperture_area,
    mag_zero_point=mag_zero_point
)

synthetic_image += sky_bg

In [9]:
plt.imshow(synthetic_image, cmap='gray')
plt.show()

In [10]:
# IMPORTANT: here we define star_catalog so that star trails can be drawn
star_catalog = get_star_catalog(
    ra_center=ra_center,
    dec_center=dec_center,
    radius=0.5,   # search radius in degrees (ensure it's big enough)
    max_mag=19.5  # limit magnitude
)
print(f"Retrieved {len(star_catalog) if star_catalog else 0} stars.")

In [11]:
# If no stars found, star_catalog might be None. Proceed only if we have data.
if star_catalog is not None:
    start_time = datetime(2025, 1, 10, 5, 0, 0, tzinfo=timezone.utc)
    end_time = start_time + timedelta(seconds=60)
    num_sub_steps = 20

    add_star_trails_skyfield(
        image=synthetic_image,
        star_catalog=star_catalog,
        wcs_obj=wcs_obj,
        lat_deg=34.05,
        lon_deg=-118.25,
        alt_m=100.0,
        start_time=start_time,
        end_time=end_time,
        num_sub_steps=num_sub_steps,
        aperture_area=aperture_area,
        quantum_efficiency=quantum_efficiency,
        mag_zero_point=mag_zero_point,
        psf_kernel=psf_kernel,
        extinction=1.3  # example atmospheric extinction
    )

In [12]:
read_noise_e = 5.0
noisy_image = np.random.poisson(synthetic_image) + np.random.normal(loc=0.0, scale=read_noise_e, size=synthetic_image.shape)
noisy_image = np.fliplr(noisy_image)

In [ ]:
plt.imshow(noisy_image, cmap='gray', vmin=np.percentile(noisy_image, 0.5), vmax=np.percentile(noisy_image, 99.5))
plt.show()

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
wcs_header["EXPTIME"] = (exposure_time, "Exposure time in seconds")
wcs_header["DATE-OBS"] = (datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S"), "UTC date/time of observation start")
wcs_header["TELESCOP"] = ("Simulator", "Telescope name")
wcs_header["PIXSCALE"] = (pixscale_arcsec, "Plate scale in arcsec/pixel")
wcs_header["RA"] = (ra_center, "Right Ascension in degrees")
wcs_header["DEC"] = (dec_center, "Declination in degrees")
wcs_header["RADECSYS"] = ("ICRS", "Right Ascension and Declination system")

hdu = fits.PrimaryHDU(data=noisy_image, header=wcs_header)
hdu.writeto(f"output/synthetic_image_{timestamp}.fits", overwrite=True)
print(f"Synthetic image saved to synthetic_image_{timestamp}.fits")